# Robust Test for $t=2$ on temporal splits
* Model: Stacking

## Setup

First, let's import a few common modules.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
import random
seed_val = 43
np.random.seed(seed_val)
random.seed(seed_val)

## Data Loading

In [ ]:
seed = 42
train_set = pd.read_csv(f"datasets/train_set_t_2_tempo.csv")


In [21]:
from sklearn.utils import shuffle

train_set = shuffle(train_set).reset_index(drop=True)

In [23]:
X = train_set.drop(['index_o', 'state_now', 'state_next_1', 'state_next_2', 'state_next_3', 'is_drop'], axis=1)
y = train_set['is_drop'].copy()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15764 entries, 0 to 15763
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               15764 non-null  int64  
 1   semester           15764 non-null  int64  
 2   grade              15764 non-null  int64  
 3   sex                15764 non-null  object 
 4   gpa_last_seme      15764 non-null  float64
 5   credits_last_seme  15764 non-null  float64
 6   credits_tot        15764 non-null  float64
 7   n_seme             15764 non-null  int64  
 8   college            15764 non-null  object 
 9   adm_unit           15764 non-null  int64  
 10  nation             15764 non-null  int64  
 11  in_capa            15764 non-null  bool   
 12  years_since        15764 non-null  int64  
 13  leave              15764 non-null  bool   
dtypes: bool(2), float64(3), int64(7), object(2)
memory usage: 1.5+ MB


## Data Transformation

In [ ]:
#from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attrs = [ 'grade', 'gpa_last_seme', 'credits_last_seme', 'credits_tot', 'n_seme', 'years_since']
cat_attrs = ['semester', 'sex', 'adm_unit', 'nation', 'in_capa', 'college', 'leave']

full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', OneHotEncoder(), cat_attrs)
])

cat_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', 'passthrough', cat_attrs)
])

## Model Evaluation


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict, cross_validate

cat_indices = [6, 7, 8, 9, 10, 11, 12]

estimators = [
    ('svm', make_pipeline(full_pipeline, SVC(C=10, gamma=0.1, random_state=42))),
    ('mlp', make_pipeline(full_pipeline, MLPClassifier(max_iter=10_000, hidden_layer_sizes=[10,10,10], random_state=42))),
    ('xgb', make_pipeline(full_pipeline, XGBClassifier(max_depth=4, n_estimators=70))),
    ('lgbm', make_pipeline(full_pipeline, LGBMClassifier(max_depth=8, n_estimators=100))),
    ('cat', make_pipeline(cat_pipeline, CatBoostClassifier(cat_features=cat_indices))),
]
stk_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(C=10)
)

In [28]:
from sklearn.metrics import (
    precision_recall_curve, recall_score, precision_score, f1_score,
    average_precision_score, roc_auc_score, matthews_corrcoef
)
from pprint import pprint

def run_model(clf, X_train, y_train): 
    res = cross_validate(clf, X_train, y_train, cv=5, scoring=('average_precision', 'roc_auc', 'recall', 'precision', 'f1', 'matthews_corrcoef'))

    auc = res['test_roc_auc'].mean()
    ap = res['test_average_precision'].mean()
    recall_no = res['test_recall'].mean()
    preci_no = res['test_precision'].mean()
    f1_no = res['test_f1'].mean()
    mcc_no = res['test_matthews_corrcoef'].mean()

    # Threshold optimization
    if (type(clf).__name__ == 'SVC'):
            y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='decision_function')
    else: 
        y_score = cross_val_predict(clf, X_train, y_train, cv=5, method='predict_proba')[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_train, y_score)

    # Maximizing F1-score
    f1_val = (2 * precision * recall) / (precision + recall)
    f1_val[np.isinf(f1_val)] = np.nan

    i = np.nanargmax(f1_val)
    best_thre = thresholds[i]

    y_pred = (y_score >= best_thre)
    recall_opt = recall_score(y_train, y_pred)
    preci_opt = precision_score(y_train, y_pred)
    f1_opt = f1_score(y_train, y_pred)
    mcc_opt = matthews_corrcoef(y_train, y_pred)

    res = {'ap': ap, 'auc': auc, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc': mcc_no, 'recall(opt)': recall_opt, 'precision(opt)': preci_opt,
           'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt, 'threshold': best_thre}
    return res

In [29]:
names = ['stk']

cvres_stk = {}

for name in names:
    clf = globals()[f'{name}_clf']
    globals()[f'cvres_{name}'] = run_model(clf, X, y)
    

[LightGBM] [Info] Number of positive: 619, number of negative: 11992
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000865 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 498
[LightGBM] [Info] Number of data points in the train set: 12611, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.049084 -> initscore=-2.963890
[LightGBM] [Info] Start training from score -2.963890
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 495, number of negative: 9593
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000693 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[

In [30]:
for name in names:
    clf = globals()[f'{name}_clf']
    print("model: ", type(clf).__name__)
    pprint(globals()[f'cvres_{name}'], sort_dicts=False)
    print()

model:  StackingClassifier
{'ap': 0.4801545218598485,
 'auc': 0.895066743012219,
 'recall': 0.26917469627147045,
 'precision': 0.6617319831431244,
 'f1': 0.3814851528553357,
 'mcc': 0.40394301783664865,
 'recall(opt)': 0.5459249676584734,
 'precision(opt)': 0.4547413793103448,
 'f1(opt)': 0.49617871840094063,
 'mcc(opt)': 0.46988107860339656,
 'threshold': 0.0905600871449238}



## Model Training and Test

In [31]:
stk_clf.fit(X, y)

[LightGBM] [Info] Number of positive: 773, number of negative: 14991
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000946 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 501
[LightGBM] [Info] Number of data points in the train set: 15764, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.049036 -> initscore=-2.964926
[LightGBM] [Info] Start training from score -2.964926
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 619, number of negative: 11992
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001300 seconds.
You can set `force_row_wise=true` to remove the

StackingClassifier(estimators=[('svm',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('num',
                                                                                  StandardScaler(),
                                                                                  ['grade',
                                                                                   'gpa_last_seme',
                                                                                   'credits_last_seme',
                                                                                   'credits_tot',
                                                                                   'n_seme',
                                                                                   'years_since']),
                                                                                 ('cat',
                                                                                  OneHotEncoder(),
                                                                                  ['semester',
                                                                                   'sex',
                                                                                   'adm_unit',
                                                                                   'nation',
                                                                                   'in_capa',
                                                                                   'college',
                                                                                   'leave'])])),
                                                ('svc',
                                                 SVC(C=10, gamma=0.1,
                                                     random_state=42...
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('num',
                                                                                  StandardScaler(),
                                                                                  ['grade',
                                                                                   'gpa_last_seme',
                                                                                   'credits_last_seme',
                                                                                   'credits_tot',
                                                                                   'n_seme',
                                                                                   'years_since']),
                                                                                 ('cat',
                                                                                  OneHotEncoder(),
                                                                                  ['semester',
                                                                                   'sex',
                                                                                   'adm_unit',
                                                                                   'nation',
                                                                                   'in_capa',
                                                                                   'college',
                                                                                   'leave'])])),
                                                ('lgbmclassifier',
                                                 LGBMClassifier(max_depth=8))]))],
                   final_estimator=LogisticRegression(C=10))

In [ ]:
test_set = pd.read_csv(f"datasets/test_set_t_2_tempo.csv")
test_set = shuffle(test_set).reset_index(drop=True)

X_te = test_set.drop(['state_now', 'state_next_1', 'state_next_2', 'state_next_3', 'is_drop'], axis=1)
y_te = test_set['is_drop'].copy()


In [34]:
len(test_set)

8748

In [ ]:
# function to evaluate the performance with test set and given threshold
def test_model(clf, X_test, y_test, thre): 
    if (type(clf).__name__ == 'SVC'):
        y_score = clf.decision_function(X_test)
    else: y_score = clf.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_score)
    ap = average_precision_score(y_test, y_score)

    if (type(clf).__name__ == 'SVC'):
        y_pred = (y_score >= 0)
    else: y_pred = (y_score >= 0.5)
    recall_no = recall_score(y_test, y_pred)
    preci_no = precision_score(y_test, y_pred)
    f1_no = f1_score(y_test, y_pred)
    mcc_no = matthews_corrcoef(y_test, y_pred)

    # Maximizing F1-score
    y_pred = (y_score >= thre)
    recall_opt = recall_score(y_test, y_pred)
    preci_opt = precision_score(y_test, y_pred)
    f1_opt = f1_score(y_test, y_pred)
    mcc_opt = matthews_corrcoef(y_test, y_pred)


    res = {'auc': auc, 'ap': ap, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc_no': mcc_no, 'recall(opt)': recall_opt, 
           'precision(opt)': preci_opt, 'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt,
           'threshold': thre}
    print("model: ", type(clf).__name__)
    pprint(res, sort_dicts=False)
    print()
    return res

In [ ]:
# Now we test the six models with the test set and given thresholds.

test_stk = {}

# set up the classifier
test_stk = test_model(stk_clf, X_te, y_te, cvres_stk['threshold'])


model:  StackingClassifier
{'auc': 0.8850535388634939,
 'ap': 0.49453139586551675,
 'recall': 0.28142589118198874,
 'precision': 0.7009345794392523,
 'f1': 0.40160642570281124,
 'mcc_no': 0.42369783814245027,
 'recall(opt)': 0.5590994371482176,
 'precision(opt)': 0.44744744744744747,
 'f1(opt)': 0.4970809007506255,
 'mcc(opt)': 0.46386348779279457,
 'threshold': 0.0905600871449238}

